In [60]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [115]:
import mlflow
import mlflow.sklearn  # Pour enregistrer les modèles scikit-learn
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import numpy as np
from mlflow.models import ModelSignature
from mlflow.types import ColSpec, Schema

In [62]:
mlflow.set_tracking_uri("http://localhost:5000")

In [63]:
pip install --user scikit-learn==1.5.1

Note: you may need to restart the kernel to use updated packages.


In [64]:
print("Version de scikit-learn :", sklearn.__version__)

Version de scikit-learn : 1.5.1


In [65]:
# Charger les données
fichier_csv = "../../data/blackjack_data.csv"  # À ajuster selon l'emplacement
data = pd.read_csv(fichier_csv)

In [66]:
# Afficher les premières lignes
print("Aperçu des données :")
print(data.head())

Aperçu des données :
   Total_Joueur  Carte_Croupier  Action
0            14              10   tirer
1             9               9   tirer
2            13               3  rester
3            20               2  rester
4            12               7   tirer


In [67]:
# Vérifier les informations générales
print("\nInformations sur les données :")
print(data.info())


Informations sur les données :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Total_Joueur    10000 non-null  int64 
 1   Carte_Croupier  10000 non-null  int64 
 2   Action          10000 non-null  object
dtypes: int64(2), object(1)
memory usage: 234.5+ KB
None


In [68]:
# Vérifier les valeurs manquantes
print("\nValeurs manquantes :")
print(data.isnull().sum())


Valeurs manquantes :
Total_Joueur      0
Carte_Croupier    0
Action            0
dtype: int64


In [69]:
# Vérifier la distribution des actions
print("\nDistribution des actions :")
print(data['Action'].value_counts())


Distribution des actions :
tirer     5178
rester    4822
Name: Action, dtype: int64


In [70]:
# Encodage de la colonne "Action"
encoder = LabelEncoder()
data['Action'] = encoder.fit_transform(data['Action'])  # "tirer" -> 1, "rester" -> 0

In [71]:
# Définir les features et le label
X = data[['Total_Joueur', 'Carte_Croupier']]  # Entrées
y = data['Action']  # Sortie

In [72]:
# Diviser en ensemble d'entraînement et de test (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
print("\nDonnées préparées :")
print(f"Taille de l'ensemble d'entraînement : {X_train.shape[0]} exemples")
print(f"Taille de l'ensemble de test : {X_test.shape[0]} exemples")


Données préparées :
Taille de l'ensemble d'entraînement : 8000 exemples
Taille de l'ensemble de test : 2000 exemples


In [116]:
mlflow.start_run()

<ActiveRun: >

In [117]:
# Initialiser et entraîner le modèle
model = DecisionTreeClassifier(max_depth=5)
model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)
# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)

In [118]:
# Enregistrer les hyperparamètres du modèle (par exemple, la profondeur maximale de l'arbre)
mlflow.log_param("max_depth", 5)

# Enregistrer les métriques (par exemple, l'accuracy)
mlflow.log_metric("accuracy", accuracy)

In [119]:
# Définir l'exemple d'entrée pour le modèle
example_input = pd.DataFrame({
    'Total_Joueur': [14],
    'Carte_Croupier': [10]
})

# Définir la signature du modèle avec Schema et ColSpec
input_schema = Schema([ColSpec("double", "Total_Joueur"), ColSpec("double", "Carte_Croupier")])
output_schema = Schema([ColSpec("integer", "Action")])

signature = ModelSignature(inputs=input_schema, outputs=output_schema)

In [120]:
mlflow.sklearn.log_model(model, "decision_tree_model", signature=signature, input_example=example_input)
# Enregistrer l'encodeur comme un artifact
# Sauvegarder l'encodeur dans un fichier temporaire
with open("encoder.pkl", "wb") as f:
    joblib.dump(encoder, f)

# Log l'encodeur en tant qu'artifact
mlflow.log_artifact("encoder.pkl")

In [121]:
# Fin de la session MLFlow
mlflow.end_run()

🏃 View run selective-wolf-351 at: http://localhost:5000/#/experiments/0/runs/8b5e6904d6d74569adf261c521106d32
🧪 View experiment at: http://localhost:5000/#/experiments/0


In [ ]:
# Évaluer les performances
print("\nÉvaluation du modèle :")
print(f"Accuracy du modèle : {accuracy}")
print("\nRapport de classification :")
print(classification_report(y_test, y_pred))

In [ ]:
# Sauvegarder le modèle dans le dossier 'ml/'
joblib.dump((model, encoder), "../model_registry/model.joblib")
print("Modèle sauvegardé avec succès !")

In [ ]:
model, encoder = joblib.load("../model_registry/model.joblib")

# Exemple de prédiction
exemple = pd.DataFrame([[15, 6]], columns=['Total_Joueur', 'Carte_Croupier'])
prediction = model.predict(exemple)

# Décoder la prédiction pour obtenir "tirer" ou "rester"
action = encoder.inverse_transform(prediction)
print("Prédiction :", action[0])

In [ ]:
import sklearn
print(sklearn.__version__)

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [100]:
# Démarrer une session MLFlow
mlflow.start_run()

<ActiveRun: >

In [101]:
# Initialiser le modèle
model_lr = LogisticRegression()

# Entraînement du modèle
model_lr.fit(X_train, y_train)

LogisticRegression()

In [102]:
# Prédiction sur l'ensemble de test
y_pred_lr = model_lr.predict(X_test)

In [103]:
# Calculer les métriques
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)

In [104]:
# Enregistrer les métriques dans MLFlow
mlflow.log_metric("accuracy", accuracy_lr)
mlflow.log_metric("precision", precision_lr)
mlflow.log_metric("recall", recall_lr)
mlflow.log_metric("f1_score", f1_lr)

In [105]:
# Définir l'exemple d'entrée pour le modèle
example_input = pd.DataFrame({
    'Total_Joueur': [14],
    'Carte_Croupier': [10]
})

# Définir la signature du modèle avec Schema et ColSpec
input_schema = Schema([ColSpec("double", "Total_Joueur"), ColSpec("double", "Carte_Croupier")])
output_schema = Schema([ColSpec("integer", "Action")])

signature = ModelSignature(inputs=input_schema, outputs=output_schema)

In [106]:
# Enregistrer le modèle avec MLFlow
mlflow.sklearn.log_model(model_lr, "logistic_regression_model", signature=signature, input_example=example_input)

In [107]:
# Fin de la session MLFlow
mlflow.end_run()

print(f"Accuracy: {accuracy_lr}")
print(f"Precision: {precision_lr}")
print(f"Recall: {recall_lr}")
print(f"F1 Score: {f1_lr}")

🏃 View run beautiful-dog-694 at: http://localhost:5000/#/experiments/0/runs/a6e7d29ca92f424eaacb2ea0649b0874
🧪 View experiment at: http://localhost:5000/#/experiments/0
Accuracy: 0.891
Precision: 0.8866666666666667
Recall: 0.903883495145631
F1 Score: 0.8951923076923077
